# Real-Time Turnstile Data Ingestion Pipeline using PySpark, Kafka, and MongoDB

This script establishes a real-time data pipeline that ingests simulated MTA subway turnstile events from Kafka, parses the raw CSV format using PySpark, and stores the structured data into:
- A **MongoDB collection** for persistent storage,
- A **console sink** for debugging, and
- An optional **in-memory table** for live SQL queries.

The pipeline ensures low-latency ETL and serves as the foundation for foot traffic prediction and analytics.


In [2]:
#!pip install pyspark

In [3]:
# Import necessary libraries
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

## Initialize Spark Session with Kafka & MongoDB Support

In [5]:
# Step 1: Create Spark Session
spark = SparkSession.builder \
    .appName("MTA_Turnstile_Streaming") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0," \
            "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")


:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/gopalakrishnaabba/.ivy2/cache
The jars for the packages stored in: /Users/gopalakrishnaabba/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-26371bfb-5195-4871-9138-d7bbc1ecc233;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#com

Creates a Spark session with Kafka and MongoDB connectors loaded, enabling structured streaming and sink capabilities.

## Define Schema for Turnstile Data

In [8]:
# Step 2: Define schema of MTA turnstile data
turnstile_schema = StructType([
    StructField("C/A", StringType(), True),
    StructField("UNIT", StringType(), True),
    StructField("SCP", StringType(), True),
    StructField("STATION", StringType(), True),
    StructField("LINENAME", StringType(), True),
    StructField("DIVISION", StringType(), True),
    StructField("DATE", StringType(), True),
    StructField("TIME", StringType(), True),
    StructField("DESC", StringType(), True),
    StructField("ENTRIES", IntegerType(), True),
    StructField("EXITS", IntegerType(), True)
])

Specifies the structure of the incoming Kafka CSV data, allowing correct parsing into DataFrame columns.

## Read Stream from Kafka Topic

In [11]:
# Step 3: Read data from Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "mta_turnstile_topic") \
    .option("startingOffsets", "latest") \
    .load()

25/05/05 11:17:11 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/05/05 11:17:11 INFO SharedState: Warehouse path is 'file:/Users/gopalakrishnaabba/spark-warehouse'.


Connects to the Kafka topic `mta_turnstile_topic` and reads incoming data as a stream.

## Parse Kafka Message Value

In [14]:
# Step 4: Parse the Kafka 'value' as CSV
turnstile_values = kafka_df.selectExpr("CAST(value AS STRING) as csv")
split_col = split(turnstile_values['csv'], ',')

Converts the Kafka byte stream to string and splits the CSV format into an array for column extraction.

## Extract Columns from CSV

In [17]:
# Step 5: Extract structured columns
turnstile_df = turnstile_values.select(
    split_col.getItem(0).alias("C/A"),
    split_col.getItem(1).alias("UNIT"),
    split_col.getItem(2).alias("SCP"),
    split_col.getItem(3).alias("STATION"),
    split_col.getItem(4).alias("LINENAME"),
    split_col.getItem(5).alias("DIVISION"),
    split_col.getItem(6).alias("DATE"),
    split_col.getItem(7).alias("TIME"),
    split_col.getItem(8).alias("DESC"),
    split_col.getItem(9).cast("integer").alias("ENTRIES"),
    split_col.getItem(10).cast("integer").alias("EXITS")
)

Maps each field in the split array to a named DataFrame column, applying proper types.

In [19]:
# Step 6: Use raw (non-aggregated) entries/exits instead of groupBy sum
raw_df = turnstile_df.select(
    "C/A", "UNIT", "SCP", "STATION", "DATE", "TIME", "DESC",
    col("ENTRIES").alias("ENTRY_COUNT"),
    col("EXITS").alias("EXIT_COUNT")
)

# Add a timestamp for ingestion time
from pyspark.sql.functions import current_timestamp
raw_df = raw_df.withColumn("ingest_time", current_timestamp())


## Sink 

- Console Sink – Print Output to Terminal: Streams the parsed data to the console in real-time. Good for debugging or quick checks.

- MongoDB Sink – Store Raw Data: Writes each micro-batch of raw turnstile data to a MongoDB collection named `raw_turnstile`.

- Memory Sink – In-Memory Table: Registers the live data as an in-memory temporary table for Spark SQL queries during streaming.

In [ ]:
# Step 7: Define output sinks

# 1. Console Sink 
query_console = raw_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .start()

# 2. MongoDB Sink

def write_to_mongo(batch_df, batch_id):
    batch_df.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/mta_db.raw_turnstile") \
        .mode("append") \
        .save()

query_mongo = raw_df.writeStream \
    .outputMode("append") \
    .foreachBatch(write_to_mongo) \
    .start()

# 3. Memory Sink (optional)
query_memory = raw_df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("live_turnstile") \
    .start()

# Wait for all queries to complete
query_console.awaitTermination()
query_mongo.awaitTermination()
query_memory.awaitTermination()


25/05/05 11:17:13 INFO StateStoreCoordinatorRef: Registered StateStoreCoordinator endpoint
25/05/05 11:17:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-bc2e5356-56e4-4957-aa2c-12ef0ace60da. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/05 11:17:13 INFO ResolveWriteToStream: Checkpoint root file:///private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-bc2e5356-56e4-4957-aa2c-12ef0ace60da resolved to file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-bc2e5356-56e4-4957-aa2c-12ef0ace60da.
25/05/05 11:17:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/05 11:17:14 INFO CheckpointF

-------------------------------------------
Batch: 0
-------------------------------------------
+---+----+---+-------+----+----+----+-----------+----------+-----------+
|C/A|UNIT|SCP|STATION|DATE|TIME|DESC|ENTRY_COUNT|EXIT_COUNT|ingest_time|
+---+----+---+-------+----+----+----+-----------+----------+-----------+
+---+----+---+-------+----+----+----+-----------+----------+-----------+



25/05/05 11:17:17 INFO CheckpointFileManager: Writing atomically to file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-bc2e5356-56e4-4957-aa2c-12ef0ace60da/offsets/1 using temp file file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-bc2e5356-56e4-4957-aa2c-12ef0ace60da/offsets/.1.955418cd-7b0a-41fd-81a1-6e8b13599e89.tmp
25/05/05 11:17:17 INFO CheckpointFileManager: Writing atomically to file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-886a90c6-9a58-417a-a60c-81be3f85b37c/offsets/1 using temp file file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-886a90c6-9a58-417a-a60c-81be3f85b37c/offsets/.1.88a8a05e-3915-4581-ab7d-4f41accc7b6c.tmp
25/05/05 11:17:17 INFO CheckpointFileManager: Writing atomically to file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-5ed78dd3-63f7-4e36-9a00-0b514cfa4dc6/offsets/1 using temp file file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/tempora

-------------------------------------------
Batch: 1
-------------------------------------------


25/05/05 11:17:18 INFO CodeGenerator: Code generated in 11.491667 ms
25/05/05 11:17:18 INFO WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 1, writer: ConsoleWriter[numRows=20, truncate=false]] committed.
25/05/05 11:17:18 INFO CodeGenerator: Code generated in 26.576041 ms
25/05/05 11:17:18 INFO DataWritingSparkTask: Writer for partition 0 is committing.
25/05/05 11:17:18 INFO DataWritingSparkTask: Committed partition 0 (task 3, attempt 0, stage 3.0)
25/05/05 11:17:18 INFO KafkaBatchReaderFactory: Creating Kafka reader topicPartition=mta_turnstile_topic-0 fromOffset=3232 untilOffset=3233, for query queryId=158f6c2b-9de3-4554-a13e-4b3f3a8337e6 batchId=1 taskId=4 partitionId=0
25/05/05 11:17:18 INFO KafkaDataConsumer: From Kafka topicPartition=mta_turnstile_topic-0 groupId=spark-kafka-source-6031fef8-68c2-46b3-b822-edab567f2440-1946334870-executor read 1 records through 1 polls (polled  out 1 records), taking 608194917 nanos, during time span of 1017974625 nanos

...

25/05/05 11:27:57 INFO BlockManagerInfo: Removed broadcast_449_piece0 on gopalas-laptop.lan:59335 in memory (size: 16.9 KiB, free: 366.2 MiB)
25/05/05 11:27:57 INFO BlockManagerInfo: Removed broadcast_450_piece0 on gopalas-laptop.lan:59335 in memory (size: 413.0 B, free: 366.2 MiB)
25/05/05 11:27:57 INFO BlockManagerInfo: Removed broadcast_454_piece0 on gopalas-laptop.lan:59335 in memory (size: 413.0 B, free: 366.2 MiB)
25/05/05 11:27:57 INFO BlockManagerInfo: Removed broadcast_451_piece0 on gopalas-laptop.lan:59335 in memory (size: 5.8 KiB, free: 366.2 MiB)
25/05/05 11:27:57 INFO BlockManagerInfo: Removed broadcast_453_piece0 on gopalas-laptop.lan:59335 in memory (size: 16.8 KiB, free: 366.3 MiB)
25/05/05 11:27:57 INFO BlockManagerInfo: Removed broadcast_448_piece0 on gopalas-laptop.lan:59335 in memory (size: 6.3 KiB, free: 366.3 MiB)
25/05/05 11:27:57 INFO BlockManagerInfo: Removed broadcast_447_piece0 on gopalas-laptop.lan:59335 in memory (size: 5.8 KiB, free: 366.3 MiB)
25/05/05 11

-------------------------------------------
Batch: 114
-------------------------------------------
+----+----+--------+---------------+----------+--------+-------+-----------+----------+-----------------------+
|C/A |UNIT|SCP     |STATION        |DATE      |TIME    |DESC   |ENTRY_COUNT|EXIT_COUNT|ingest_time            |
+----+----+--------+---------------+----------+--------+-------+-----------+----------+-----------------------+
|R023|R045|00-00-00|34 ST-HERALD SQ|05/05/2025|11:27:59|REGULAR|8          |9         |2025-05-05 11:28:01.006|
+----+----+--------+---------------+----------+--------+-------+-----------+----------+-----------------------+



25/05/05 11:28:06 INFO CheckpointFileManager: Writing atomically to file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-bc2e5356-56e4-4957-aa2c-12ef0ace60da/offsets/115 using temp file file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-bc2e5356-56e4-4957-aa2c-12ef0ace60da/offsets/.115.540aa400-9212-463d-ad1e-8a5eaa62a9d6.tmp
25/05/05 11:28:06 INFO CheckpointFileManager: Writing atomically to file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-5ed78dd3-63f7-4e36-9a00-0b514cfa4dc6/offsets/115 using temp file file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-5ed78dd3-63f7-4e36-9a00-0b514cfa4dc6/offsets/.115.fcf11886-c75e-4b40-b033-d36badfe1528.tmp
25/05/05 11:28:06 INFO CheckpointFileManager: Writing atomically to file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn/T/temporary-886a90c6-9a58-417a-a60c-81be3f85b37c/offsets/115 using temp file file:/private/var/folders/48/hk2s0t4x1xv5gpns58fpsx1m0000gn

-------------------------------------------
Batch: 115
-------------------------------------------
+----+----+--------+---------------+----------+--------+-------+-----------+----------+-----------------------+
|C/A |UNIT|SCP     |STATION        |DATE      |TIME    |DESC   |ENTRY_COUNT|EXIT_COUNT|ingest_time            |
+----+----+--------+---------------+----------+--------+-------+-----------+----------+-----------------------+
|R023|R045|00-00-00|34 ST-HERALD SQ|05/05/2025|11:28:05|REGULAR|5          |14        |2025-05-05 11:28:06.384|
+----+----+--------+---------------+----------+--------+-------+-----------+----------+-----------------------+



25/05/05 11:28:09 INFO BlockManagerInfo: Removed broadcast_458_piece0 on gopalas-laptop.lan:59335 in memory (size: 413.0 B, free: 366.2 MiB)
25/05/05 11:28:09 INFO BlockManagerInfo: Removed broadcast_461_piece0 on gopalas-laptop.lan:59335 in memory (size: 16.9 KiB, free: 366.2 MiB)
25/05/05 11:28:09 INFO BlockManagerInfo: Removed broadcast_459_piece0 on gopalas-laptop.lan:59335 in memory (size: 6.3 KiB, free: 366.2 MiB)
25/05/05 11:28:09 INFO BlockManagerInfo: Removed broadcast_460_piece0 on gopalas-laptop.lan:59335 in memory (size: 5.8 KiB, free: 366.2 MiB)
25/05/05 11:28:09 INFO BlockManagerInfo: Removed broadcast_456_piece0 on gopalas-laptop.lan:59335 in memory (size: 6.3 KiB, free: 366.2 MiB)
25/05/05 11:28:09 INFO BlockManagerInfo: Removed broadcast_462_piece0 on gopalas-laptop.lan:59335 in memory (size: 413.0 B, free: 366.2 MiB)
25/05/05 11:28:09 INFO BlockManagerInfo: Removed broadcast_457_piece0 on gopalas-laptop.lan:59335 in memory (size: 16.8 KiB, free: 366.3 MiB)
25/05/05 11